<a href="https://colab.research.google.com/github/otoperalias/Coyuntura/blob/main/clases/Tema3_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tema 3.2. Desestacionalización (I)
Leer las diapositivas disponibles en el Aula Virtual antes de ejecutar este código.

In [ ]:
#Importamos algunas librerías
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
plt.style.use('seaborn-v0_8-whitegrid')

#### La importancia de desestacionalizar

In [ ]:
# Importamos datos de personas ocupadas (miles) de la CNTR
ocup=pd.read_excel("https://github.com/otoperalias/Coyuntura/raw/main/clases/datos/T3_ocupados_cntr.xlsx", index_col=0,parse_dates=True)
# Calculamos tasa de variación intertrimestral
ocup_tv=(ocup/ocup.shift(1)-1)*100

In [ ]:
# Evolución de los ocupados (datos brutos)
fig,ax=plt.subplots()
ax.plot(ocup_tv.loc["2005":"2016",'ocup_naj'], lw=2.5, color="teal", alpha=0.7)
ax.set_ylabel("Tasa var intertrim (%)\n")
ax.set_xlabel("")
ax.set_title("Evolución de los ocupados en España\n", size=10, weight="bold")
ax.axhline(0, color="black")
ax.spines[["bottom","top","right","left"]].set_visible(False)
ax.grid(axis="x")
fig.text(0.1,0,"Fuente: CNTR (INE). Elaboración propia.")
plt.show()

In [ ]:
# Evolución de los ocupados (datos ajustados)
fig,ax=plt.subplots()
ax.plot(ocup_tv.loc["2005":"2016",'ocup_aj'], lw=2.5, color="teal", alpha=0.7)
ax.set_ylabel("Tasa var intertrim (%)\n")
ax.set_xlabel("")
ax.set_title("Evolución de los ocupados en España\n", weight="bold")
ax.axhline(0, color="black")
ax.spines[["bottom","top","right","left"]].set_visible(False)
ax.grid(axis="x")
fig.text(0.1,-0.05,"Fuente: Datos ajustados por estacionalidad y calendario CNTR (INE).\nElaboración propia.")
plt.show()

#### Descomposición clásica

In [ ]:
#Importamos datos de pasajeros de aerolíneas (en miles)
series = pd.read_csv('https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/airline-passengers.csv', header=0, index_col=0, parse_dates=True)
series

In [ ]:
# Representar la serie
series.plot()

In [ ]:
# Ejemplo descomposición aditiva
result_ad = sm.tsa.seasonal_decompose(series, model="additive")
result_ad.plot()
plt.show()

In [ ]:
# Obtenemos serie ajustada
serieaj=series.Passengers-result_ad.seasonal

In [ ]:
fig,ax=plt.subplots()
ax.plot(series.Passengers)
ax.plot(serieaj)
plt.show()

In [ ]:
# Ejemplo de descomposición multiplicativa
result = sm.tsa.seasonal_decompose(series, model='multiplicative')
result.plot()
plt.show()

In [ ]:
# Obtenemos serie ajustada
serieajm=series.Passengers/result.seasonal

In [ ]:
fig,ax=plt.subplots()
ax.plot(series.Passengers)
ax.plot(serieajm)
plt.show()

#### Descomposición STL

In [ ]:
# Echar un vistazo a la página de ayuda:
# https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.STL.html#statsmodels.tsa.seasonal.STL

#Ahora aplicamos el nuevo método a la serie de pasajeros de aerolíneas
result = sm.tsa.STL(series).fit()
result.plot()
plt.show()

In [ ]:
# Para reducir la influencia de los valores atípicos, se añade el argumento robust (no siempre es mejor o conveniente):
result = sm.tsa.STL(series, robust=True).fit()
result.plot()
plt.show()

In [ ]:
serieaj_stl=series.Passengers-result.seasonal

In [ ]:
fig,ax=plt.subplots()
ax.plot(series.Passengers, label="Original")
ax.plot(serieaj_stl,label="STL")
ax.plot(serieaj,label="Aditivo")
ax.plot(serieajm,label="Multiplicativo")
ax.legend()
plt.show()

#### Material adicional:

Para profundizar en vuestros conocimientos, echad un vistazo a este gran recurso disponible en la web:  
http://www.chadfulton.com/topics/002-seasonal-adjustment.html

#### Ejemplo con datos de afiliaciones a la Seguridad Social

En este ejercicio, vamos a descargar los datos de afiliados medios a la Seguridad Social para España, todos los periodos, ambos sexos, total edad y "Todos los regímenes", "REGIMEN GENERAL", "R.G.(S.E.AGRARIO)" y "REG. ESP. AUTONOMOS". A la hora de guardar en excel, pivotamos la tabla de manera que en las filas aparezcan los periodos y en las columnas las cuatro series de afiliados.

Fuente:  
https://w6.seg-social.es/PXWeb/pxweb/es/

In [ ]:
# Importamos los datos
ss=pd.read_csv("Afiliados medios_20251027-104508.csv", sep=";",skiprows=1 ,encoding="latin_1")
ss

In [ ]:
ss.columns

In [ ]:
# Procesamos los datos
# (Podemos reusar (adaptándolo) el código disponible en https://github.com/otoperalias/Coyuntura/blob/main/clases/Tema2_IV.ipynb)

# Primero: convertimos la columna en string
ss['Periodo']=ss['Periodo'].astype(str)
# Segundo: le damos formato
ss['Periodo']=ss['Periodo'].str[:4]+"-"+ss['Periodo'].str[-2:]
# Tercero: convertimos en datetime
ss['Periodo']=pd.to_datetime(ss['Periodo'])
# Cuarto: convertimos en index.
ss.set_index("Periodo", inplace=True)
# Renombramos columnas
ss.rename(columns={'Total Sexo Total Edad Todos los Regimenes':"total",
                   'Total Sexo Total Edad REGIMEN GENERAL':"general",
                   'Total Sexo Total Edad R.G.(S.E.AGRARIO)':"campo",
                   'Total Sexo Total Edad REG. ESP. AUTONOMOS':"auto"},inplace=True)


In [ ]:
ss.plot()

In [ ]:
# Descomposición clásica aditiva
result = sm.tsa.seasonal_decompose(ss["total"], model="additive")
ss_ad=ss["total"]-result.seasonal
result.plot()
plt.show()

In [ ]:
# Descomposición clásica multiplicativa
result = sm.tsa.seasonal_decompose(ss["total"], model="multiplcative")
ss_mul=ss["total"]/result.seasonal
result.plot()
plt.show()

In [ ]:
# Descomposición STL
result = sm.tsa.STL(ss["total"], robust=True).fit()
ss_stl=ss["total"]-result.seasonal
result.plot()
plt.show()

In [ ]:
# GRÁFICO CON LA SERIE ORIGINAL Y LA AJUSTADA CON LOS TRES MÉTODOS
fig,ax=plt.subplots()
ax.plot(ss.total, label="Original")
ax.plot(ss_ad,label="Aditivo")
ax.plot(ss_mul,label="Multiplicativo")
ax.plot(ss_stl,label="STL")
ax.legend()
plt.show()

In [ ]:
# Gráfico para ver en qué meses hay más y menos afiliados
result = sm.tsa.seasonal_decompose(ss["total"], model="multiplcative")
fig,ax=plt.subplots()
result.seasonal["2020"].plot(ax=ax,color="teal",linewidth=4, alpha=0.7) # da igual el año que se elija (siempre igual en la desc. clásica)
ax.axhline(1,color="black")
plt.show()

In [ ]:
# GRÁFICO MÁS PROFESIONAL
# a. Preparamos los datos
import numpy as np
result = sm.tsa.seasonal_decompose(ss["general"], model="multiplcative")
df=(result.seasonal["2020"]-1)*100  # da igual el año que se elija (siempre igual en la desc. clásica)
meses_num=np.arange(0,12)
meses=["Ene","Feb","Mar","Abr","Mayo","Jun","Jul","Ago","Sep","Oct","Nov","Dec"]
# a. Creamos el gráfico
fig,ax=plt.subplots()
df.plot.bar(ax=ax,color="tab:red")
ax.set_xticks(meses_num)
ax.set_xticklabels(meses, rotation='horizontal')
ax.axhline(0,color="black")
ax.set_ylabel("Variación porcentual respecto\n a la media anual (%)")
ax.set_xlabel("")
ax.set_title("Variación mensual en la afiliación a la Seguridad Social", size=14, pad=35)
fig.text(0.5,0.92,"Variación respecto a la media anual", size=12,ha="center")
fig.text(0.13,-0.05,"Periodo analizado: 01/2012-09/25.\nFuente: Seguridad Social. Elaboración propia.")
plt.show()